# Experimentation

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
DATA_FILEPATH = Path("data", "input", "air_quality_health_impact_data.csv")
air = pd.read_csv(DATA_FILEPATH)

## Exploratory Data Analysis
This section is dedicated to understanding the data, identifying and correcting missing values, and examing distributions of features. Additionally, we will look at the covariance and correlation matrices to see if, visually, we can identify any basic trends in the data.

In [ ]:
print(f"Observations: {air.shape[0]}")
print(f"Features: {air.shape[1]}")
air.head()

In [ ]:
air.describe(include="all")

In [ ]:
air.isna().sum()

In [ ]:
bins = 20

fig, axs = plt.subplots(7, 2, figsize=(12, 16))

axs[0, 0].hist(air["AQI"], bins=bins)
axs[0, 0].set_ylabel("Count")
axs[0, 0].set_title("Distribution of AQI")

axs[0, 1].hist(air["PM10"], bins=bins)
axs[0, 1].set_ylabel("Count")
axs[0, 1].set_title("Distribution of PM10")

axs[1, 0].hist(air["PM2_5"], bins=bins)
axs[1, 0].set_ylabel("Count")
axs[1, 0].set_title("Distribution of PM2_5")

axs[1, 1].hist(air["NO2"], bins=bins)
axs[1, 1].set_ylabel("Count")
axs[1, 1].set_title("Distribution of NO2")

axs[2, 0].hist(air["SO2"], bins=bins)
axs[2, 0].set_ylabel("Count")
axs[2, 0].set_title("Distribution of SO2")

axs[2, 1].hist(air["O3"], bins=bins)
axs[2, 1].set_ylabel("Count")
axs[2, 1].set_title("Distribution of O3")

axs[3, 0].hist(air["Temperature"], bins=bins)
axs[3, 0].set_ylabel("Count")
axs[3, 0].set_title("Distribution of Temperature")

axs[3, 1].hist(air["Humidity"], bins=bins)
axs[3, 1].set_ylabel("Count")
axs[3, 1].set_title("Distribution of Humidity")

axs[4, 0].hist(air["WindSpeed"], bins=bins)
axs[4, 0].set_ylabel("Count")
axs[4, 0].set_title("Distribution of WindSpeed")

axs[4, 1].hist(air["RespiratoryCases"], bins=bins)
axs[4, 1].set_ylabel("Count")
axs[4, 1].set_title("Distribution of RespiratoryCases")

axs[5, 0].hist(air["CardiovascularCases"], bins=bins)
axs[5, 0].set_ylabel("Count")
axs[5, 0].set_title("Distribution of CardiovascularCases")

axs[5, 1].hist(air["HospitalAdmissions"], bins=bins)
axs[5, 1].set_ylabel("Count")
axs[5, 1].set_title("Distribution of HospitalAdmissions")

axs[6, 0].hist(air["HealthImpactScore"], bins=bins)
axs[6, 0].set_ylabel("Count")
axs[6, 0].set_title("Distribution of HealthImpactScore")

axs[6, 1].hist(air["HealthImpactClass"], bins=bins)
axs[6, 1].set_ylabel("Count")
axs[6, 1].set_title("Distribution of HealthImpactClass")

plt.tight_layout()
plt.show()

Let's display the covariance matrix of the dataset. The "covariance" measures how changes in one variable are associated with changes in a second variable. In other words, the covariance measures the degree to which two variables are linearly associated.

In [ ]:
air.cov(numeric_only=True)

A few examples of what we can find from the data here include:

1. The high covariance (286.94) of AQI and PM10 indicates that a higher AQI is associated with a higher level of particulates 10 µm or smaller.
2. The extremely high covariance (1185.04) between AQI and HealthImpactScore indicates that higher AQI is associated with a higher Health Impact Score.
3. The negative covariance between O3 and NO2 (-74.46) indicates that higher Ozone is associated with lower levels of nitrogen dioxide.

Let's now display the correlation matrix. "Correlation" measures both the strength and direction of the linear relationship between two variables:


In [ ]:
air.corr(numeric_only=True)

## Preprocessing

This section will define and utilize functions to split the data into training, validation, and test splits. Additionally, we will create preprocessing steps to scale the data by removing the mean and dividing by the standard deviation. This returns us a feature centered around 0, making learning easier.

### Function Declarations

In [ ]:
import joblib
import tarfile
import tempfile
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer


def preprocess(base_directory, job_type="classification"):
    """Load the supplied data, split it and transform it."""
    df = _read_data_from_input_csv_files(base_directory)

    if job_type == "classification":
        target_transformer = ColumnTransformer(
            transformers=[
                ("Class", FunctionTransformer(np.abs), ["HealthImpactClass"])
            ],
        )
    else:
        target_transformer = ColumnTransformer(
            transformers=[("Score", StandardScaler(), ["HealthImpactScore"])],
        )

    numeric_transformer = make_pipeline(
        SimpleImputer(strategy="mean"),
        StandardScaler(),
    )

    features_transformer = ColumnTransformer(
        transformers=[
            (
                "numeric",
                numeric_transformer,
                make_column_selector(dtype_exclude="object"),
            ),
        ],
    )

    df_train, df_validation, df_test = _split_data(df)

    y_train = target_transformer.fit_transform(df_train)
    y_validation = target_transformer.transform(df_validation)
    y_test = target_transformer.transform(df_test)

    _save_train_baseline(base_directory, df_train)
    _save_test_baseline(base_directory, df_test)

    df_train = df_train.drop("HealthImpactClass", axis=1)
    df_validation = df_validation.drop("HealthImpactClass", axis=1)
    df_test = df_test.drop("HealthImpactClass", axis=1)

    df_train = df_train.drop("HealthImpactScore", axis=1)
    df_validation = df_validation.drop("HealthImpactScore", axis=1)
    df_test = df_test.drop("HealthImpactScore", axis=1)

    X_train = features_transformer.fit_transform(df_train)  # noqa: N806
    X_validation = features_transformer.transform(df_validation)  # noqa: N806
    X_test = features_transformer.transform(df_test)  # noqa: N806

    _save_splits(
        base_directory,
        X_train,
        y_train,
        X_validation,
        y_validation,
        X_test,
        y_test,
    )
    _save_model(base_directory, target_transformer, features_transformer)


def _read_data_from_input_csv_files(base_directory):
    """Read the data from the input CSV files.

    This function reads every CSV file available and
    concatenates them into a single dataframe.
    """
    input_directory = Path(base_directory) / "input"
    files = list(input_directory.glob("*.csv"))

    if len(files) == 0:
        message = f"The are no CSV files in {input_directory.as_posix()}/"
        raise ValueError(message)

    raw_data = [pd.read_csv(file) for file in files]
    df = pd.concat(raw_data)

    # Shuffle the data
    return df.sample(frac=1, random_state=42)


def _split_data(df):
    """Split the data into train, validation, and test."""
    df_train, temp = train_test_split(df, test_size=0.3)
    df_validation, df_test = train_test_split(temp, test_size=0.5)

    return df_train, df_validation, df_test


def _save_train_baseline(base_directory, df_train):
    """Save the untransformed training data to disk.

    We will need the training data to compute a baseline to
    determine the quality of the data that the model receives
    when deployed.
    """
    baseline_path = Path(base_directory) / "train-baseline"
    baseline_path.mkdir(parents=True, exist_ok=True)

    df = df_train.copy().dropna()

    # To compute the data quality baseline, we don't need the
    # target variable, so we'll drop it from the dataframe.
    df = df.drop("HealthImpactClass", axis=1)
    df = df.drop("HealthImpactScore", axis=1)

    df.to_csv(baseline_path / "train-baseline.csv", header=True, index=False)


def _save_test_baseline(base_directory, df_test):
    """Save the untransformed test data to disk.

    We will need the test data to compute a baseline to
    determine the quality of the model predictions when deployed.
    """
    baseline_path = Path(base_directory) / "test-baseline"
    baseline_path.mkdir(parents=True, exist_ok=True)

    df = df_test.copy().dropna()

    # We'll use the test baseline to generate predictions later,
    # and we can't have a header line because the model won't be
    # able to make a prediction for it.
    df.to_csv(baseline_path / "test-baseline.csv", header=False, index=False)


def _save_splits(
    base_directory,
    X_train,  # noqa: N803
    y_train,
    X_validation,  # noqa: N803
    y_validation,
    X_test,  # noqa: N803
    y_test,
):
    """Save data splits to disk.

    This function concatenates the transformed features
    and the target variable, and saves each one of the split
    sets to disk.
    """

    train = np.concatenate((X_train, y_train), axis=1)
    validation = np.concatenate((X_validation, y_validation), axis=1)
    test = np.concatenate((X_test, y_test), axis=1)

    train_path = Path(base_directory) / "train"
    validation_path = Path(base_directory) / "validation"
    test_path = Path(base_directory) / "test"

    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(
        validation_path / "validation.csv",
        header=False,
        index=False,
    )
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)


def _save_model(base_directory, target_transformer, features_transformer):
    """Save the Scikit-Learn transformation pipelines.

    This function creates a model.tar.gz file that
    contains the two transformation pipelines we built
    to transform the data.
    """
    with tempfile.TemporaryDirectory() as directory:
        joblib.dump(target_transformer, Path(directory) / "target.joblib")
        joblib.dump(features_transformer, Path(directory) / "features.joblib")

        model_path = Path(base_directory) / "model"
        model_path.mkdir(parents=True, exist_ok=True)

        with tarfile.open(f"{(model_path / 'model.tar.gz').as_posix()}", "w:gz") as tar:
            tar.add(Path(directory) / "target.joblib", arcname="target.joblib")
            tar.add(
                Path(directory) / "features.joblib",
                arcname="features.joblib",
            )

In [ ]:
base_directory = Path("data")
preprocess(base_directory, job_type="classification")

## Model Training

This section will define and test the model using the sklearn library. Different models will be used, and different hyperparameters will be tested in order to try to find the best configuration for performance.

### XGBoost
This first section will examine XGBoost, a popular machine learning framework for regression and classification tasks.

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

train_path = Path(base_directory, "train")
X_train = pd.read_csv(Path(train_path, "train.csv"), header=None)
y_train = X_train[X_train.columns[-1]]
X_train = X_train.drop(X_train.columns[-1], axis=1)

val_path = Path(base_directory, "validation")
X_val = pd.read_csv(Path(val_path, "validation.csv"), header=None)
y_val = X_val[X_val.columns[-1]]
X_val = X_val.drop(X_val.columns[-1], axis=1)

# Define the hyperparameters and their search ranges
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 4, 5],
    "min_child_weight": [1, 3, 5],
    "subsample": [0.8, 0.9, 1.0],
    "colsample_bytree": [0.8, 0.9, 1.0],
}

# Create an XGBoost model
xgb_model = xgb.XGBClassifier()

# Perform GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)

# Perform GridSearchCV
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Fit the model with the best hyperparameters on the entire dataset
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10)

# Evaluate the best model on the test set
accuracy = best_model.score(X_val, y_val)
print(f"Best Hyperparameters: {best_params}")
print(f"Accuracy on test set: {accuracy:.2f}")